In [ ]:
# Idea here is: 
# - Load the training dataset that we previously generated
# - Transform all rules into single rules
    # Remember to drop duplicates here!
# - Select a number of data ~ the other datasets (for fairness)

In [1]:
# Load root folder of the Git project
from pathlib import Path
import subprocess

repo_root = Path(
    subprocess.check_output(
        ["git", "rev-parse", "--show-toplevel"],
        text=True
    ).strip()
)

In [2]:
from core.data_extraction.utils import create_hash_rule_sentence, create_positive_sample_id

def create_single_rules(df):
    df['concepts'] = df['concepts'].apply(lambda el: [el])
    hashed_dataset = create_hash_rule_sentence(df, "target", "concepts")
    positive_samples_df = create_positive_sample_id(hashed_dataset, "concepts") 
    positive_samples_df['concepts'] = positive_samples_df['concepts'].apply(lambda el: el[0])
    positive_samples_df = positive_samples_df.drop_duplicates(ignore_index=True)
    return positive_samples_df   

In [3]:
import os 
import pandas as pd
from tqdm import tqdm 
INPUT_PATH = repo_root / "offline_datasets/processed_dataset/CommonNet/kfold_division/"
for k_fold in tqdm(os.listdir(INPUT_PATH)):
    if "fold" in k_fold:
        full_path = os.path.join(INPUT_PATH, k_fold)
        train_df = pd.read_parquet(os.path.join(full_path, "train.parquet"))
        single_rule_train_df = create_single_rules(train_df)
        single_rule_train_df.to_parquet(os.path.join(full_path, "single_rule_train.parquet"))

        validation_df = pd.read_parquet(os.path.join(full_path, "validation.parquet"))
        single_rule_validation_df = create_single_rules(validation_df)
        single_rule_validation_df.to_parquet(os.path.join(full_path, "single_rule_validation.parquet"))

100%|██████████| 11/11 [01:13<00:00,  6.72s/it]
